# Performing Molecular Dynamics - Rotaxane

Just the MD code for the Multi-Molecule System Tutorial Rotaxane that we built.

Once we have all the output files from CHARMM-GUI, we can use these in OpenMM to perform our simulation.
The remainder of the code has nothing to do with BuildAMol anymore. It's now on to OpenMM coding!

In [2]:
import openmm
import openmm.app as app
import openmm.unit as unit

In [13]:
from pathlib import Path
charmm_downloads = Path("./files/charmm-gui-rotaxane")

In [14]:
# load the pdb of our molecule (it will be named "ligandrm.pdb" by CHARMM-GUI)
pdb = app.PDBFile(str(charmm_downloads / "ligandrm.pdb"))

# load the psf of our molecule (it will be named "ligandrm.psf" by CHARMM-GUI)
psf = app.CharmmPsfFile(str(charmm_downloads / "ligandrm.psf"))

# load the CHARMM force field parameters 
# we need to include the CGenFF parameters as well...
params = app.CharmmParameterSet(
                                str(charmm_downloads / "toppar" / "lig.rtf"),
                                str(charmm_downloads / "toppar" / "lig.prm"),
                                str(charmm_downloads / "toppar" / "top_all36_cgenff.rtf"),
                                str(charmm_downloads / "toppar" / "par_all36_cgenff.prm"),
                            )

# create the OpenMM system from the PSF and parameters
# that we will need to run the simulation
system = psf.createSystem(params, 
                        nonbondedMethod=app.CutoffNonPeriodic, 
                        nonbondedCutoff=1.0*unit.nanometer, 
                        constraints=app.HBonds, 
                        )

In [15]:
# now set up some other simulation stuff that is necessary...
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(psf.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# and run the simulation
simulation.reporters.append(app.PDBReporter('files/system_MD.pdb', 10))
simulation.reporters.append(app.StateDataReporter('files/system_MD.log', 10, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)